# Operation Fusion

Using the typical order of operations rules, we evaluate the expression in parentheses first `(cos(C) / D)`, followed by the multiply, then the assignment. Written using standard C++ operator overloading, we would have a cosine, division, multiplication, and assignment overload. Each operator performs their respective task, then returns the value computed. That returned value is stored somewhere (either out to memory or possible in a register), then the next operator uses that output as input into its own computation. Finally, the assignment writes the value, usually out to memory.

To avoid this overhead, MatX uses a technique called lazy evaluation to reduce the total number of loads and stores. It does this by overloading each operator so that **instead of performing the operation, such as multiplication, instead it returns an object that represents multiplication when it’s needed.** The entire expression is generates a single type in C++ representing the equation above, and when we ask for element (0,0) of A above, the value is computed on-the-fly without storing any values. This also implies that you can store an entire expression into a variable and nothing will be exectuted:

In [ ]:
//todo this should be moved to a hidden init block that runs automatically when the notebook starts
#pragma cling add_library_path("/usr/local/cuda/lib64")
#pragma cling add_library_path("/opt/xeus/cling/lib")
//#pragma cling add_library_path("/usr/Lib/gcc/x86_64-Linux-gnu/11/")
#pragma cling add_library_path("/usr/lib/x86_64-linux-gnu/openblas64-openmp/")
#pragma cling add_include_path("/usr/local/cuda/include")
#pragma cling add_include_path("/usr/include/x86_64-linux-gnu/openblas64-openmp")
#pragma cling add_include_path("/opt/xeus/cling/tools/Jupyter/kernel/MatX/include")
#pragma cling add_include_path("/opt/xeus/cling/tools/Jupyter/kernel/MatX/build/_deps/cccl-src/libcudacxx/include")
//#pragma cling load("libgomp")
#pragma cling load("libopenblas64")
#pragma cling load("libcuda")
#pragma cling load("libcudart")
#pragma cling load("libcurand")
#pragma cling load("libcublas")
#pragma cling load("libcublasLt")

#include <cuda/std/__algorithm/max.h>
#include <cuda/std/__algorithm/min.h>

#define MATX_EN_OPENBLAS
#define MATX_EN_OPENBLAS_LAPACK
#define MATX_OPENBLAS_64BITINT

#include "matx.h"

exec = matx::SingleThreadedHostExecutor;

In [ ]:
auto op = (B * (cos(C) / D));

In the example above op can be further combined with other expressions, which can increase code readability without loss of performance.

## TODO Add Timeline showing set of ops and when they actually execute on the GPU

## Fusion Example

In [ ]:
size_t size_x = 128;
size_t size_y = 256;

auto A      = matx::make_tensor({size_x, size_y});
auto B      = matx::make_tensor({size_x, size_y});
auto C      = matx::make_tensor({size_x, size_y});
auto D      = matx::make_tensor({size_x, size_y});
auto result = matx::make_tensor({size_x, size_y});

// first C++ code
(result = A + B).run(exec);       // read A, read B,      write result
(result = result + C).run(exec);  // read C, read result, write result
(result += D).run(exec);          // read D, read result, write result
// total memory accesses: read: 6 * size_x * size_y, write: 3 * size_x * size_y

(result = A + B + C + D).run(exec);
// total memory accesses: read: 4 * size_x * size_y, write: 1 * size_x * size_y
// ~1/2 memory accesses compared to the previous code


### Runtime Improvements
The fused results in all of the performance benefits we described above:
- a single kernel is submitted to the GPU to complete all operations
- memory is only read from global once and written to global once

# TODO show Exectutor timing of  how the fusion improved performance


this results in X less memory traffic and Y faster kernel execution.

## Fusion with Operators

Fusion is intuitive when all operands can be combined into a single statement, however fusion can also provide significant benefit for readability and reuse in implementations where very complex terms are defined, or even reused in the same solution. 

due to the lazy evaluation of operators, operator terms can be defined to clearly construct the specfic math for each term, and then these operators can be combined later to create the complete final expression for execution.

Below we show a more complex operation comprised of both unary operators and transforms, and how we can break down a very complex expression into simple terms that can be reused

# TODO: this is currently nonsense. do we want to update this to be more sensical?


In [ ]:

// Implementation 1: All terms together
(result = A * C  + conv1d(B) / D + ((D - C) / B) / A * C).run(exec); 


// Implemenatation 2: break each term into it's own operation for readability
(result = A*C).run(exec);
(result += conv1d(B) / D).run(exec);
(result += (D - C) / B).run(exec);

// Implementation 3: create operators for each term, and then combine in single execution
auto term1 = A * C; 
auto term2 = conv1d(B) / D;
auto term3 = (D - C) / B;
auto term4 = term3 / term1;
(result = term1 + term2 + term4).run(exec);

### Runtime Improvements

again we see the performance improvments of fusing the independent terms together, however we now also have the programming and readability benefit of code with well-defined terms.

# TODO: include exectuion performance

## Limitations & Intermediates
some limitions exist that prevents the fusion of all operations. Like all CUDA programs, there is an upper ceiling to the complexity of how much compute is optimal for a given kernel, as the kernel's complexity drives resource utilization (such as registers and shared memory), that my ultimately harm performance.

Similarly some lower-level APIs utilized by MatX may not support iterators / pre / post operations, and cannot be fused. 

To resolve this MatX uses Asnychronous memory when required to create intermediate outputs to store information between non-fusable operations. This does not require any action on the user to enable, however it may result in sub-optimal performance if asnchronous pools are not managed appropriately.  

In [ ]:
// dims don't work as is
// outline when we need to write output for sake of API (FFT, not sure what else?)
result = (ifft(matmul(B, fft(A))) + C ) * D; 

# Exercise: Black Scholes Fusion

The Black Scholes model provides a fantastic example of a real-world set of equations that greatly benefits from operator fusion. Black Scholes provides both a complex set of expressions that provide significant readability improvements if expressed as individual expressions, but also benefits from fusion of its separate operational parts. Below is a brief description of the Black Scholes models and its composite terms:


$$
C(S_0, K, T) = S_0 \,\Phi\bigl(d_1\bigr) \;-\; K \, e^{-rT} \,\Phi\bigl(d_2\bigr),
$$

where

$$
d_1 = \frac{\ln\!\bigl(\tfrac{S_0}{K}\bigr) + \bigl(r + \tfrac{\sigma^2}{2}\bigr)T}{\sigma \sqrt{T}},
\quad
d_2 = d_1 - \sigma \sqrt{T}.
$$

Here:
- \( S_0 \) is the current stock price
- \( K \) is the strike price
- \( T \) is the time to maturity (in years)
- \( r \) is the risk-free interest rate (annualized)
- \( \sigma \) is the volatility of the underlying stock (annualized)
- \( \Phi(\cdot) \) is the cumulative distribution function (CDF) of the standard normal distribution



We can easily translate this by expressing each of the terms defined above as separate MatX operators, then fusing the execution of those operators in the final run command.

Try breaking the equation below into the following operators:

```
VsqrtT  = V * sqrt(T);
d1      = (log(S / K) + (r + 0.5 * V * V) * T) / VsqrtT ;
d2      = d1 - VsqrtT;
cdf_d1  = normcdf(d1);
cdf_d2  = normcdf(d2);
expRT   = exp(-1 * r * T); 
```


In [ ]:
using dtype = double;
index_t input_size = 10;
// index_t inputIsze  = 10000000; // increase size to measure performance

//declare input data
auto K = matx::make_tensor<dtype>({input_size});
auto S = matx::make_tensor<dtype>({input_size});
auto V = matx::make_tensor<dtype>({input_size});
auto r = matx::make_tensor<dtype>({input_size});
auto T = matx::make_tensor<dtype>({input_size});
auto output = matx::make_tensor<dtype>({input_size});  
auto referenceOutput = matx::make_tensor<dtype>({input_size});  

// Individually Evaluated Reference
(referenceOutput = S * matx::normcdf((matx::log(S / K) + (r + 0.5 * V * V) * T) / V * matx::sqrt(T)) - K * matx::exp(-1 * r * T) * matx::normcdf((matx::log(S / K) + (r + 0.5 * V * V) * T) / V * sqrt(T) - V * sqrt(T);)).run(exec);
print(referenceOutput);

// well organized version
// auto VsqrtT = V * sqrt(T);
// auto d1     = (log(S / K) + (r + 0.5 * V * V) * T) / VsqrtT ;
// auto d2     = d1 - VsqrtT;
// auto cdf_d1 = normcdf(d1);
// auto cdf_d2 = normcdf(d2);
// auto expRT  = exp(-1 * r * T); 

// (output = S * cdf_d1 - K * expRT * cdf_d2).run(exec);


